In [5]:
import argparse
import os
import os.path as op
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio 
import nipype.interfaces.utility as niu 
import nipype.interfaces.ants as ants 
from niworkflows.interfaces.bids import DerivativesDataSink

def main(derivatives, ds, mask_type='accolla'):
    if mask_type == 'pca':
        mask_template = op.join(derivatives, ds, 'masks_warped', 'sub-{subject}', 'sub-{subject}_space-T1w_desc-{mask}_subroi-{subroi}.nii.gz')
        subrois = ['A', 'B', 'C']
    else:
        mask_template = op.join(derivatives, ds, 'masks_warped_accolla', 'sub-{subject}', 'sub-{subject}_space-T1w_desc-{mask}-{subroi}_mask.nii.gz')
        subrois = ['limbic', 'associative', 'motor']
    
    if ds == 'ds-01':
        subjects = ['{:02d}'.format(s) for s in range(1, 20)]
    elif ds == 'ds-02':
        subjects = ['{:02d}'.format(s) for s in range(1, 16)]
        subjects.pop(3) # Remove 4


    templates = {'preproc': op.join(derivatives, ds, 'fmriprep', 'sub-{subject}', 'func', 'sub-{subject}_task-randomdotmotion_run-*_space-T1w_desc-preproc_bold.nii.gz'),
                 'individual_mask': mask_template,
                 }

    wf = pe.Workflow(name='extract_signal_submasks_{}_{}'.format(ds, mask_type))
    wf.base_dir = os.path.join(derivatives, '..', 'processing', 'nipype_workflow_folders')
    wf.config = {"execution": {"crashdump_dir": os.path.join(derivatives, '..', 'processing', 'crashdumps')}}

    mask_identity = pe.Node(niu.IdentityInterface(fields=['mask', 'subroi']),
                            name='mask_identity')
    mask_identity.iterables = [('mask', ['stnl', 'stnr']), ('subroi', subrois)]

    selector = pe.Node(nio.SelectFiles(templates),
                       name='selector')

    selector.iterables = [('subject', subjects)]
    wf.connect(mask_identity, 'mask', selector, 'mask')
    wf.connect(mask_identity, 'subroi', selector, 'subroi')

    def extract_signal(preproc, mask):
        from nilearn import image
        from nilearn import input_data
        from nipype.utils.filemanip import split_filename
        import os.path as op
        import pandas as pd

        _, fn, ext = split_filename(preproc)
        masker = input_data.NiftiMasker(mask, standardize='psc')

        data = pd.DataFrame(masker.fit_transform(preproc))

        new_fn = op.abspath('{}_signal.csv'.format(fn))
        data.to_csv(new_fn)

        return new_fn

    extract_signal_node = pe.MapNode(niu.Function(function=extract_signal,
                                     input_names=['preproc', 'mask'],
                                     output_names=['signal']),
                         iterfield=['preproc'],
                        name='extract_signal_node')

    wf.connect(selector, 'preproc', extract_signal_node, 'preproc')
    wf.connect(selector, 'individual_mask', extract_signal_node, 'mask')

    ##  throws an error that it cannot create the path, can't be bothered to fix
#     datasink_signal = pe.MapNode(DerivativesDataSink(base_directory=op.join(derivatives, ds),
#                                                      out_path_base='extracted_signal', 
#                                                      allowed_entities=("custom",)),
#                                  iterfield=['source_file', 'in_file'], 
#                                 name='datasink_signal') 

#     wf.connect(selector, 'preproc', datasink_signal, 'source_file')
#     wf.connect(extract_signal_node, 'signal', datasink_signal, 'in_file')
#     wf.connect(mask_identity, 'mask', datasink_signal, 'desc')
 
#     def get_subroi_suffix(subroi):
#         return 'subroi-{}'.format(subroi)

#     wf.connect(mask_identity, ('subroi', get_subroi_suffix), datasink_signal, 'suffix')

    datasink = pe.Node(nio.DataSink(), name='datasink')    #, iterfield=['signal'])
    datasink.inputs.base_directory = os.path.join(derivatives, ds, 'extracted_signal')
    datasink.inputs.regexp_substitutions = [('signal/_mask_([a-z]{4})_subroi_([A-Za-z]+)/_subject_([0-9]{2})/.*/sub-[0-9]{2}_task-randomdotmotion_run-0([0-9])_space-T1w_desc-preproc_bold_signal.csv',
                                             'sub-\\3/sub-\\3_task-randomdotmotion_run-\\4_desc-\\1_subroi-\\2_roi.csv')
                                             ]
    wf.connect(extract_signal_node, 'signal', datasink, 'signal')


    wf.run(plugin='MultiProc',
           plugin_args={'n_procs': 10})
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('ds', type=str,)
    parser.add_argument('mask_type', type=str,)
    args = parser.parse_args()

    main('/home/stevenm/Projects/bias/derivatives',
         args.ds, args.mask_type)

usage: ipykernel_launcher.py [-h] ds mask_type
ipykernel_launcher.py: error: the following arguments are required: mask_type


SystemExit: 2

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
